Extract tickers obtaineed from ssm.ipynb

In [ ]:
from pprint import pprint

ticker_names = []

with open(r'./subset_c.txt', 'r') as fp:
    for line in fp:
        x = line[:-1] # excluding \n char
        ticker_names.append(x)

pprint(ticker_names)

Rank the target stock list by descending dividend yield.

In [ ]:
import yfinance as yf

tickers = list(yf.Tickers(ticker_names).tickers.values())

# pprint(tickers[0].info)
cnt = 0
for ticker in tickers:
    try:
        ticker.info['dividendYield']
    except KeyError:
        tickers.remove(ticker)

sorted_tickers = sorted(tickers, key=lambda x: x.info["dividendYield"], reverse=True)
print(sorted_tickers[0].info['dividendYield'])

For each target stock get the PUT option chain with expiry dates more than 3 days but less than 15 days before the ex-div date ["exDividendDate"].

In [ ]:
from datetime import datetime
tickers = []

for ticker in sorted_tickers:
    ticker_dates = {}
    ticker_dates["ticker"] = ticker
    ticker_dates["options"] = []
    exDividendDate = datetime.fromtimestamp(ticker.info["exDividendDate"]).date()
    ticker_dates["exDividendDate"] = exDividendDate
    for option in ticker.options:
        date = datetime.strptime(option, '%Y-%m-%d').date() 
        difference = exDividendDate - date
        if 3 < difference.days < 15:
            ticker_dates["options"].append(option)
    if len(ticker_dates["options"]) > 0:
        tickers.append(ticker_dates)

pprint(tickers)

Remove all contracts with an open interest less than 1000 contracts.
Remove all contracts with a strike price greater than or less than 2% of the current market price.  

In [ ]:
import pandas as pd

for ticker in tickers:
    ticker['contracts'] = pd.DataFrame()
    market_price = ticker["ticker"].history()['Close'].iloc[-1]
    print(f"MARKET PRICE: {market_price}")
    for option in ticker["options"]:
        chain = ticker["ticker"].option_chain(option)
        puts_calls = pd.concat([chain.puts, chain.calls])
        puts_calls.query('`openInterest` > 1000', inplace=True)
        puts_calls.query('`strike` > (@market_price * 0.98)', inplace=True)
        puts_calls.query('`strike` < @market_price * 1.02', inplace=True)
        ticker['contracts'] = pd.concat([ticker['contracts'], puts_calls])
    
pprint(tickers)



Rank the contracts by descending premiums.